In [ ]:
# Install Ray with support for the dashboard + cluster launcher
# run: install_pkg.sh

# Install Ray with minimal dependencies
# !pip install -U ray

# !pip install -U "ray[air]"     # installs Ray + dependencies for Ray AI Runtime
# !pip install -U "ray[tune]"    # installs Ray + dependencies for Ray Tune
# !pip install -U "ray[rllib]"   # installs Ray + dependencies for Ray RLlib
# !pip install -U "ray[serve]"   # installs Ray + dependencies for Ray Serve

In [1]:
import ray
import warnings
import math
import os

# Suppress noisy requests warnings.
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [2]:
ray.init()

2022-12-14 14:12:38,871	INFO worker.py:1352 -- Connecting to existing Ray cluster at address: 10.3.27.163:6379...
2022-12-14 14:12:38,881	INFO worker.py:1538 -- Connected to Ray cluster.


Python version:,3.8.10
Ray version:,2.2.0


In [3]:
direct = '../PySpark/taxi_data.parquet/'
list_files = os.listdir(direct)
filter_files = []

for file in list_files:
    if file[-8:] == '.parquet':
        filter_files.append(direct + file)
        
# Read Parquet files in parallel
ds = ray.data.read_parquet(filter_files)

RayTaskError(FileNotFoundError): [36mray::_get_read_tasks()[39m (pid=424007, ip=10.3.27.163)
  File "/usr/local/lib/python3.8/dist-packages/ray/data/read_api.py", line 1586, in _get_read_tasks
    reader = ds.create_reader(**kwargs)
  File "/usr/local/lib/python3.8/dist-packages/ray/data/datasource/parquet_datasource.py", line 165, in create_reader
    return _ParquetDatasourceReader(**kwargs)
  File "/usr/local/lib/python3.8/dist-packages/ray/data/datasource/parquet_datasource.py", line 203, in __init__
    _handle_read_os_error(e, paths)
  File "/usr/local/lib/python3.8/dist-packages/ray/data/datasource/file_meta_provider.py", line 367, in _handle_read_os_error
    raise error
  File "/usr/local/lib/python3.8/dist-packages/ray/data/datasource/parquet_datasource.py", line 199, in __init__
    pq_ds = pq.ParquetDataset(
  File "/usr/local/lib/python3.8/dist-packages/pyarrow/parquet/core.py", line 1724, in __new__
    return _ParquetDatasetV2(
  File "/usr/local/lib/python3.8/dist-pac

In [5]:
ds.schema()

vendor_id: string
pickup_datetime: timestamp[ns]
dropoff_datetime: timestamp[ns]
passenger_count: int32
trip_distance: double
pickup_longitude: double
pickup_latitude: double
rate_code: int32
store_and_fwd_flag: string
dropoff_longitude: double
dropoff_latitude: double
payment_type: string
fare_amount: double
surcharge: double
mta_tax: double
tip_amount: double
tolls_amount: double
total_amount: double
-- schema metadata --
org.apache.spark.version: '3.3.1'
org.apache.spark.sql.parquet.row.metadata: '{"type":"struct","fields":[{"' + 1283

In [59]:
total_rows = ds.count()
print('Total rows: {:,}'.format(total_rows))

Total rows: 15,811,595


In [9]:
# Display dataframe
ds.limit(5).to_pandas()

Read progress: 100%|██████████| 1/1 [00:00<00:00, 895.64it/s]


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,CMT,2012-03-29 18:07:30,2012-03-29 18:33:42,1,2.60,-73.972451,40.754851,1,N,-73.985300,40.768561,CSH,13.7,1.0,0.5,0.0,0.0,15.2
1,CMT,2012-03-30 15:30:10,2012-03-30 15:36:13,1,2.00,-73.954359,40.766108,1,N,-73.971284,40.744712,CSH,6.9,0.0,0.5,0.0,0.0,7.4
2,CMT,2012-03-30 08:11:45,2012-03-30 08:17:33,1,0.80,-73.953557,40.771037,1,N,-73.961918,40.777928,CSH,4.9,0.0,0.5,0.0,0.0,5.4
3,CMT,2012-03-31 11:55:39,2012-03-31 12:00:41,1,0.60,-73.985258,40.761061,1,N,-73.979457,40.762585,CSH,4.5,0.0,0.5,0.0,0.0,5.0
4,VTS,2012-03-12 20:18:00,2012-03-12 20:20:00,1,1.01,-73.957805,40.765215,1,None,-73.950527,40.775135,CSH,4.5,0.5,0.5,0.0,0.0,5.5


In [28]:
ds.take(1)

[ArrowRow({'vendor_id': 'CMT',
           'pickup_datetime': Timestamp('2012-03-29 18:07:30'),
           'dropoff_datetime': Timestamp('2012-03-29 18:33:42'),
           'passenger_count': 1,
           'trip_distance': 2.6,
           'pickup_longitude': -73.972451,
           'pickup_latitude': 40.754851,
           'rate_code': 1,
           'store_and_fwd_flag': 'N',
           'dropoff_longitude': -73.9853,
           'dropoff_latitude': 40.768561,
           'payment_type': 'CSH',
           'fare_amount': 13.7,
           'surcharge': 1.0,
           'mta_tax': 0.5,
           'tip_amount': 0.0,
           'tolls_amount': 0.0,
           'total_amount': 15.2})]

In [44]:
def convert_size(size_bytes):
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   return "%s %s" % (s, size_name[i])

total_bytes = ds.size_bytes()
print("Total size: {}".format(convert_size(total_bytes)))

Total size: 1.99 GB


In [46]:
# In order to get the in-memory size, we can trigger full 
# reading of the dataset and inspect the size in bytes

total_bytes = ds.fully_executed().size_bytes()
print("Total in-memory size: {}".format(convert_size(total_bytes)))


Total in-memory size: 1.99 GB


In [49]:
# What's the longets trip distance, largest tip amount, and most number of passengers?
ds.max(["trip_distance", "tip_amount", "passenger_count"])

Shuffle Reduce: 100%|██████████| 1/1 [00:00<00:00, 79.57it/s]


ArrowRow({'max(trip_distance)': 100.0,
          'max(tip_amount)': 200.0,
          'max(passenger_count)': 208})

In [50]:
# Drop some columns.
ds = ds.drop_columns(["store_and_fwd_flag", "mta_tax"])

Map_Batches: 100%|██████████| 23/23 [00:01<00:00, 15.57it/s]


In [51]:
ds.limit(5).to_pandas()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,tip_amount,tolls_amount,total_amount
0,CMT,2012-03-29 18:07:30,2012-03-29 18:33:42,1,2.60,-73.972451,40.754851,1,-73.985300,40.768561,CSH,13.7,1.0,0.0,0.0,15.2
1,CMT,2012-03-30 15:30:10,2012-03-30 15:36:13,1,2.00,-73.954359,40.766108,1,-73.971284,40.744712,CSH,6.9,0.0,0.0,0.0,7.4
2,CMT,2012-03-30 08:11:45,2012-03-30 08:17:33,1,0.80,-73.953557,40.771037,1,-73.961918,40.777928,CSH,4.9,0.0,0.0,0.0,5.4
3,CMT,2012-03-31 11:55:39,2012-03-31 12:00:41,1,0.60,-73.985258,40.761061,1,-73.979457,40.762585,CSH,4.5,0.0,0.0,0.0,5.0
4,VTS,2012-03-12 20:18:00,2012-03-12 20:20:00,1,1.01,-73.957805,40.765215,1,-73.950527,40.775135,CSH,4.5,0.5,0.0,0.0,5.5


In [53]:
%%time
ds.groupby("passenger_count").count().take()

Shuffle Reduce: 100%|██████████| 23/23 [00:00<00:00, 694.28it/s]

CPU times: user 350 ms, sys: 154 ms, total: 504 ms
Wall time: 1min 4s


[PandasRow({'passenger_count': 0, 'count()': 335328}),
 PandasRow({'passenger_count': 1, 'count()': 11051985}),
 PandasRow({'passenger_count': 2, 'count()': 2242606}),
 PandasRow({'passenger_count': 3, 'count()': 666836}),
 PandasRow({'passenger_count': 4, 'count()': 312434}),
 PandasRow({'passenger_count': 5, 'count()': 1148624}),
 PandasRow({'passenger_count': 6, 'count()': 389102}),
 PandasRow({'passenger_count': 208, 'count()': 8})]

In [54]:
# Filter our records with negative passenger counts.
ds = ds.map_batches(lambda df: df[df["passenger_count"] > 0])

Map_Batches: 100%|██████████| 23/23 [00:02<00:00, 11.05it/s]


In [55]:
# Mean trip distance grouped by passenger count.
ds.groupby("passenger_count").mean("trip_distance").take()

Shuffle Reduce: 100%|██████████| 23/23 [00:00<00:00, 709.61it/s]


[PandasRow({'passenger_count': 1, 'mean(trip_distance)': 2.8091539203138622}),
 PandasRow({'passenger_count': 2, 'mean(trip_distance)': 3.022558545727604}),
 PandasRow({'passenger_count': 3, 'mean(trip_distance)': 2.9073175263483075}),
 PandasRow({'passenger_count': 4, 'mean(trip_distance)': 2.9382245850323594}),
 PandasRow({'passenger_count': 5, 'mean(trip_distance)': 2.9500852933597073}),
 PandasRow({'passenger_count': 6, 'mean(trip_distance)': 2.9708847294539726}),
 PandasRow({'passenger_count': 208, 'mean(trip_distance)': 0.0})]

## Advanced Aside - Projection and Filter Pushdown

Note that Ray Datasets' Parquet reader supports projection (column selection) and row filter pushdown, where we can push the above column selection and the row-based filter to the Parquet read. If we specify column selection at Parquet read time, the unselected columns won't even be read from disk!

The row-based filter is specified via
[Arrow's dataset field expressions](https://arrow.apache.org/docs/6.0/python/generated/pyarrow.dataset.Expression.html#pyarrow.dataset.Expression). See the {ref}`feature guide for reading Parquet data <dataset_supported_file_formats>` for more information.

In [57]:
# Only read the passenger_count and trip_distance columns.
import pyarrow as pa
filter_expr = (
    (pa.dataset.field("passenger_count") <= 10)
    & (pa.dataset.field("passenger_count") > 0)
)

pushdown_ds = ray.data.read_parquet(filter_files,
    columns=["passenger_count", "trip_distance"],
    filter=filter_expr,
)

# Force full execution of both of the file reads.
pushdown_ds = pushdown_ds.fully_executed()

Parquet Files Sample: 100%|██████████| 2/2 [00:00<00:00, 96.86it/s]
2022-12-14 12:45:28,872	WARNING read_api.py:326 -- ⚠️  The number of blocks in this dataset (23) limits its parallelism to 23 concurrent tasks. This is much less than the number of available CPU slots in the cluster. Use `.repartition(n)` to increase the number of dataset blocks.
Read progress: 100%|██████████| 23/23 [00:00<00:00, 721.77it/s]


In [61]:
total_rows = pushdown_ds.count()
print('Total rows: {:,}'.format(total_rows))

Total rows: 15,811,587


pushdown_ds.limit(10).to_pandas()

In [64]:
# Delete the pushdown dataset. Deleting the Dataset object
# will release the underlying memory in the cluster.
del pushdown_ds

## Ingesting into Model Trainers

Now that we've learned more about our data and we have cleaned up our dataset a bit, we now look at how we can feed this dataset into some dummy model trainers.

First, let's do a full global random shuffle of the dataset to decorrelate these samples.

In [66]:
ds = ds.random_shuffle()

Shuffle Reduce: 100%|██████████| 23/23 [00:01<00:00, 18.48it/s]


* We define a dummy ``Trainer`` actor, where each trainer will consume a dataset shard in batches and simulate model training.

*In a real training workflow, we would feed ``ds`` to {ref}`Ray Train <train-docs>`, which would do this sharding and creation of training actors for us, under the hood.*

In [67]:
@ray.remote
class Trainer:
    def __init__(self, rank: int):
        pass

    def train(self, shard: ray.data.Dataset) -> int:
        for batch in shard.iter_batches(batch_size=256):
            pass
        return shard.count()

trainers = [Trainer.remote(i) for i in range(4)]
trainers

[Actor(Trainer, 8716ad277846ff83b245e4a601000000),
 Actor(Trainer, c769ef29f2540a35e181a47601000000),
 Actor(Trainer, 1b6d4f8c0d584d419d969f6101000000),
 Actor(Trainer, 63d051ee100ba2f70fa40c8101000000)]

* Next, we split the dataset into ``len(trainers)`` shards, ensuring that the shards are of equal size.

In [68]:
shards = ds.split(n=len(trainers), equal=True)
shards

[Dataset(num_blocks=6, num_rows=3952898, schema={vendor_id: object, pickup_datetime: datetime64[ns], dropoff_datetime: datetime64[ns], passenger_count: int32, trip_distance: float64, pickup_longitude: float64, pickup_latitude: float64, rate_code: int32, dropoff_longitude: float64, dropoff_latitude: float64, payment_type: object, fare_amount: float64, surcharge: float64, tip_amount: float64, tolls_amount: float64, total_amount: float64}),
 Dataset(num_blocks=7, num_rows=3952898, schema={vendor_id: object, pickup_datetime: datetime64[ns], dropoff_datetime: datetime64[ns], passenger_count: int32, trip_distance: float64, pickup_longitude: float64, pickup_latitude: float64, rate_code: int32, dropoff_longitude: float64, dropoff_latitude: float64, payment_type: object, fare_amount: float64, surcharge: float64, tip_amount: float64, tolls_amount: float64, total_amount: float64}),
 Dataset(num_blocks=7, num_rows=3952898, schema={vendor_id: object, pickup_datetime: datetime64[ns], dropoff_datetim

* Finally, we simulate training, passing each shard to the corresponding trainer. The number of rows per shard is returned.

In [69]:
ray.get([w.train.remote(s) for w, s in zip(trainers, shards)])

[3952898, 3952898, 3952898, 3952898]

## Parallel Batch Inference

Refer to the blog on [Model Batch Inference in Ray](https://www.anyscale.com/blog/model-batch-inference-in-ray-actors-actorpool-and-datasets) for an overview of batch inference strategies in Ray and additional examples.

After we've trained a model, we may want to perform batch (offline) inference on such a tabular dataset. With Ray Datasets, this is as easy as a {meth}`ds.map_batches() <ray.data.Dataset.map_batches>` call!

First, we define a callable class that will cache the loading of the model in its constructor.

In [70]:
import pandas as pd

def load_model():
    # A dummy model.
    def model(batch: pd.DataFrame) -> pd.DataFrame:
        return pd.DataFrame({"score": batch["passenger_count"] % 2 == 0})
    
    return model

class BatchInferModel:
    def __init__(self):
        self.model = load_model()
    def __call__(self, batch: pd.DataFrame) -> pd.DataFrame:
        return self.model(batch)

In [71]:
ds.map_batches(BatchInferModel, batch_size=2048, compute="actors").take()

Map Progress (19 actors 5 pending): 100%|██████████| 23/23 [00:08<00:00,  2.67it/s]


[PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': True}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': True}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': True}),
 PandasRow({'score': True}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False})]

In [75]:
ds.map_batches(
    BatchInferModel,
    batch_size=256,
    #num_gpus=1,  # Uncomment this to run this on GPUs!
    compute="actors",
).take()

Map Progress (214 actors 54 pending):  57%|█████▋    | 13/23 [00:24<00:12,  1.23s/it]

(scheduler +35m15s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


Map Progress (214 actors 54 pending):  87%|████████▋ | 20/23 [00:29<00:02,  1.46it/s]

(scheduler +35m20s) Error: No available node types can fulfill resource request {'CPU': 1.0, 'GPU': 1.0}. Add suitable node types to this cluster to resolve this issue.


Map Progress (214 actors 54 pending): 100%|██████████| 23/23 [00:29<00:00,  1.30s/it]


[PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': True}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': True}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': True}),
 PandasRow({'score': True}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False})]

(scheduler +35m55s) Error: No available node types can fulfill resource request {'CPU': 1.0, 'GPU': 1.0}. Add suitable node types to this cluster to resolve this issue.
(scheduler +36m30s) Error: No available node types can fulfill resource request {'CPU': 1.0, 'GPU': 1.0}. Add suitable node types to this cluster to resolve this issue.
(scheduler +37m5s) Error: No available node types can fulfill resource request {'CPU': 1.0, 'GPU': 1.0}. Add suitable node types to this cluster to resolve this issue.
(scheduler +37m40s) Error: No available node types can fulfill resource request {'CPU': 1.0, 'GPU': 1.0}. Add suitable node types to this cluster to resolve this issue.
(scheduler +38m15s) Error: No available node types can fulfill resource request {'CPU': 1.0, 'GPU': 1.0}. Add suitable node types to this cluster to resolve this issue.


* We can also configure the autoscaling actor pool that this inference stage uses, setting upper and lower bounds on the actor pool size, and even tweak the batch prefetching vs. inference task queueing tradeoff.

In [73]:
from ray.data import ActorPoolStrategy

# The actor pool will have at least 2 workers and at most 8 workers.
strategy = ActorPoolStrategy(min_size=2, max_size=8)

ds.map_batches(
    BatchInferModel,
    batch_size=256,
    #num_gpus=1,  # Uncomment this to run this on GPUs!
    compute=strategy,
).take()

Map Progress (8 actors 0 pending): 100%|██████████| 23/23 [00:33<00:00,  1.44s/it]


[PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': True}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': True}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': True}),
 PandasRow({'score': True}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False}),
 PandasRow({'score': False})]